In [1]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from xgboost import plot_importance

pd.set_option("display.max_rows",None)

In [2]:

load_dotenv(r"/home/bdcmartinez/Desktop/Projects/ML_END_TO_END/notebooks/.env")

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


## Get the data from the database

In [3]:
df_products = pd.read_sql("SELECT * FROM products;", engine)

In [4]:
df_sales = pd.read_sql("SELECT * FROM sales;", engine)

In [5]:
df_regions = pd.read_sql("SELECT * FROM regions", engine)

## Join tables to see the results

In [6]:
df_sales_joined = df_sales[["region_id","date","quantity","payment_method"]].merge(df_regions[["id","state_name", "city"]], left_on = "region_id", right_on = "id", how="left").copy()
df_sales_joined.drop(columns=["region_id","id"], inplace=True) #we drop unnesseary values 



## Explore the data to gain insights

- What are the states with higher sales?
- What is the most sold product?
- How many products were sold in each state?

In [7]:
# import seaborn as sns
# import matplotlib.pyplot as plt


# df = df_last_months[df_last_months["city"]==4].copy()

# sns.boxplot(x=df['total_quantity'])
# plt.show()


In [8]:
# we create a month_name and year column

df_sales_joined['month_name'] = df_sales_joined['date'].dt.month_name()
df_sales_joined['year'] = df_sales_joined['date'].dt.year


In [9]:


# we focus on the sales that do not belong to the current month
df_last_months = df_sales_joined[df_sales_joined["date"]<'2025-11-01']



In [10]:
df_last_months.head()

,date,quantity,payment_method,state_name,city,month_name,year
0,2023-01-01,33.0,Credit Card,Alabama,Bessemer,January,2023
1,2023-01-01,32.0,Cash,Alabama,Bessemer,January,2023
2,2023-01-01,19.0,Cash,Alabama,Bessemer,January,2023
3,2023-01-01,37.0,Credit Card,Alabama,Bessemer,January,2023
4,2023-01-01,34.0,Credit Card,Alabama,Bessemer,January,2023


In [11]:

month_order = ['January', 'February', 'March', 'April', 'May', 'June','July', 'August', 'September', 'October', 'November', 'December']


In [12]:
def transformer(dataframe):
    df = dataframe.copy()

    # we transform the month_name column into category type
    df = (df.groupby(["year","month_name", "city"])["quantity"].sum()).reset_index()
    
    df['month_name'] = pd.Categorical(df['month_name'], categories=month_order, ordered=True)
    
    df.rename(columns={"quantity":"total_quantity"}, inplace=True)
    df = df.sort_values(["city","year","month_name"], ascending=True).reset_index(drop=True)


    #--we create the main features
    df["last_12m"] = (df.groupby("city")["total_quantity"].rolling(12).sum().reset_index(level=0, drop=True))
    df["last_6m"] = (df.groupby("city")["total_quantity"].rolling(6).sum().reset_index(level=0, drop=True))
    df["last_3m"] = (df.groupby("city")["total_quantity"].rolling(3).sum().reset_index(level=0, drop=True))
    df['lag_1'] = (df.groupby('city')['total_quantity'].shift(1))
    df['lag_2'] = (df.groupby('city')['total_quantity'].shift(2))
    df['lag_3'] = (df.groupby('city')['total_quantity'].shift(3))
    df = df.dropna() #we drop the null values obtained by the last operations


    df['year'] = df['year'].astype(str)
    df['month_name'] = df['month_name'].astype(str)
    df['date'] = pd.to_datetime(df['month_name'] + '-' + df['year'], format="%B-%Y")
    df["quarter"] = df["date"].dt.quarter


    #we convert the columns to the correct data type to train the ML model
    
    df["year"] = df["year"].astype(int)
    df["total_quantity"] = df["total_quantity"].astype(int)
    df["last_12m"] = df["last_12m"].astype(int)
    df["last_6m"] = df["last_6m"].astype(int)
    df["last_3m"] = df["last_3m"].astype(int)
    df["lag_1"] = df["lag_1"].astype(int)
    df["lag_2"] = df["lag_2"].astype(int)
    df["lag_3"] = df["lag_3"].astype(int)


    return df

In [13]:
df_full_sales = transformer(df_sales_joined)

In [14]:
#we obtain the current total quantities for the current month 
df_current_month = df_full_sales[(df_full_sales["month_name"]=="October") & (df_full_sales["year"]==2025)].copy().reset_index(drop=True)

In [15]:
df_last_months = df_full_sales[df_full_sales["date"]<'2025-11-01'].copy()


## Model training

In [16]:
target = "total_quantity"

features = [
    'year', 'city', 'last_12m', 'last_6m', 'last_3m',
    'lag_1', 'lag_2', 'lag_3'
]


In [17]:
train = df_last_months[df_last_months["year"] < 2025]
test  = df_last_months[df_last_months["year"] == 2025]

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]


In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder


numeric_features = ['year', 'last_12m', 'last_6m', 'last_3m',
                    'lag_1', 'lag_2', 'lag_3']

categorical_features = ['city']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # we use an encoder for the categoy
    ]
)


In [19]:
from xgboost import XGBRegressor

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(
        objective='reg:squarederror',
        random_state=42
    ))
])


In [20]:
param_distributions = {
    "model__n_estimators": [100, 200, 300, 500],
    "model__max_depth": [3, 4, 5, 6, 8],
    "model__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "model__subsample": [0.7, 0.8, 0.9, 1.0],
    "model__colsample_bytree": [0.6, 0.8, 1.0],
    "model__gamma": [0, 1, 5],
    "model__reg_lambda": [0.1, 1, 10],
}


In [21]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'model__colsample_bytree': [0.6, 0.8, ...], 'model__gamma': [0, 1, ...], 'model__learning_rate': [0.01, 0.05, ...], 'model__max_depth': [3, 4, ...], ...}"
,n_iter,30
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [22]:
print("Best parameters:", search.best_params_)

best_model = search.best_estimator_

y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)


Best parameters: {'model__subsample': 0.9, 'model__reg_lambda': 10, 'model__n_estimators': 500, 'model__max_depth': 4, 'model__learning_rate': 0.2, 'model__gamma': 5, 'model__colsample_bytree': 0.6}
MAE: 243.49871826171875
RMSE: 386.89939341513576


In [23]:
    # Assuming 'model' is your trained machine learning model

import joblib
joblib.dump(best_model, '/home/bdcmartinez/Desktop/Projects/ML_END_TO_END/model/model.pkl')

['/home/bdcmartinez/Desktop/Projects/ML_END_TO_END/model/model.pkl']

In [26]:
test = joblib.load('/home/bdcmartinez/Desktop/Projects/ML_END_TO_END/model/model.pkl')


In [30]:
test.predict(df_current_month[0:1])[0]

np.float32(11994.194)

In [45]:
test.predict(pd.DataFrame([{
    'city':  'Bayou Cane',
    'year' : 2025,
    'total_quantity':  12,
    'last_12m' : 1000,
    'last_6m' : 12,
    'last_3m' : 12,
    'lag_1' : 12,
    'lag_2' : 12,
    'lag_3' : 12}]))

array([10229.326], dtype=float32)

In [46]:
df_current_month.iloc[0]

year                             2025
month_name                    October
city                             Ames
total_quantity                  12420
last_12m                       138133
last_6m                         73411
last_3m                         37146
lag_1                           12040
lag_2                           12686
lag_3                           11944
date              2025-10-01 00:00:00
quarter                             4
Name: 0, dtype: object

In [24]:

for i, row in df_current_month.iterrows():

    # city name is already readable
    city_name = row["city"]

    # model takes the raw row (pipeline preprocesses automatically)
    prediction = best_model.predict(df_current_month.iloc[i:i+1])[0]

    print(city_name, prediction)


Ames 11994.194
Amsterdam 12445.611
Anacortes 12130.24
Annandale 12898.772
Aurora 12470.165
Baileys Crossroads 12010.18
Baldwin 12089.163
Batavia 12391.747
Baton Rouge 12044.063
Bay Shore 12187.428
Bayou Cane 11951.283
Bear 11970.688
Bellmore 12481.624
Bensonhurst 11858.109
Bessemer 12539.384
Bethpage 11938.252
Binghamton 11497.189
Birmingham 12241.403
Blacksburg 11894.247
Bon Air 12444.654
Borough Park 11538.043
Bossier City 12492.466
Briarwood 12501.367
Brighton 22070.598
Brighton Beach 12027.437
Bristol 11710.87
Broomfield 12381.23
Buffalo 12489.394
Casper 12058.519
Castlewood 11974.235
Cave Spring 12334.468
Cañon City 12217.003
Centennial 11694.118
Central 11739.076
Central Islip 13280.271
Centreville 11930.204
Cheektowaga 12425.659
Cherry Hill 13025.715
Chinatown 12799.289
Cicero 12041.541
Clay 11822.261
Cohoes 12353.3955
Colorado Springs 12537.143
Columbine 12105.996
Commerce City 11545.294
Coney Island 12321.742
Coram 11366.273
Daphne 12470.813
Deer Park 11685.467
Denver 12618.05

In [25]:
df_sales_g_m.tail(10)

NameError: name 'df_sales_g_m' is not defined

In [ ]:
for i, item in df_current_month.iterrows():

    encoded_value = item["city"]  # example
    original_city = ordinal_encoder.inverse_transform([[encoded_value]])[0][0]

    
    prediction = model.predict(df_current_month[features].iloc[i:i+1])

    
    print(original_city, prediction)